In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

import numpy as np

import datetime as dt
from datetime import datetime

from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats
from functools import reduce

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [202]:
path = "C:\\Users\\lexic\\OneDrive - HKUST Connect\\DatabaseX\\FF\\"

In [203]:
alex = pd.read_csv(path + 'alex_sharpe.csv')
alex['RETX'] = np.where(alex['RETX']=='C', 0, alex['RETX'])
alex['RETX'] = alex['RETX'].astype(float)
alex['date'] = alex['date'].astype(str)
alex['date'] = pd.to_datetime(alex['date'])

alex = alex.drop_duplicates(subset=['PERMNO', 'date'], keep='first')
df = alex.pivot(index='date', columns='PERMNO', values=['RETX'])

In [204]:
df.columns = ['HAS', 'RJR']
df = df.reset_index()
df['date'] = df['date'] + MonthEnd(0)

In [205]:
df = df[(df['date']>='2000-01-01') & (df['date']<='2017-06-30') ]

In [206]:
ff3 = pd.read_csv(path + 'ff3.csv', header=0, index_col=0)
ff3.index.name = 'date'
ff3 = ff3.apply(lambda x: x/100)
ff3 = ff3.reset_index()
ff3['date'] = ff3['date'].astype(str)
ff3['date'] = ff3['date'].apply(lambda x: datetime.strptime(x, "%Y%m")) + MonthEnd(0)
ff3 = ff3.rename(columns={'Mkt-RF' : 'MF'})

In [221]:
mom = pd.read_csv(path + 'mom.csv', header=0, index_col=0)
mom.index.name = 'date'
mom = mom.apply(lambda x: x/100)
mom = mom.reset_index()
mom = mom.rename(columns={'Mom   ': 'MOM'})
mom['date'] = mom['date'].astype(str)
mom['date'] = mom['date'].apply(lambda x: datetime.strptime(x, "%Y%m")) + MonthEnd(0)

In [222]:
df1 = pd.merge(df, ff3, how='left', on=['date'])
df2 = pd.merge(df1, mom, how='left', on=['date'])

In [223]:
df2['V500'] = df2['MF'] + df2['RF']

In [224]:
df2['HAS_ex'] = df2['HAS'] - df2['RF']
df2['RJR_ex'] = df2['RJR'] - df2['RF']

In [225]:
df3 = df2[(df2['date']>='2002-01-31') & (df2['date']<='2006-12-31')]

In [236]:
df2.to_csv(path + 'alex_sharpe_updated.csv')
df2

,date,HAS,RJR,MF,SMB,HML,RF,MOM,V500,HAS_ex,RJR_ex
0,2000-01-31,-0.207921,-0.024823,-0.0474,0.0579,-0.0189,0.0041,0.0192,-0.0433,-0.212021,-0.028923
1,2000-02-29,0.050000,0.047273,0.0245,0.2148,-0.0981,0.0043,0.1820,0.0288,0.045700,0.042973
2,2000-03-31,0.059524,-0.055556,0.0520,-0.1729,0.0823,0.0047,-0.0683,0.0567,0.054824,-0.060256
3,2000-04-30,-0.044944,0.220588,-0.0640,-0.0663,0.0725,0.0046,-0.0839,-0.0594,-0.049544,0.215988
4,2000-05-31,0.027451,0.337349,-0.0442,-0.0611,0.0483,0.0050,-0.0898,-0.0392,0.022451,0.332349
...,...,...,...,...,...,...,...,...,...,...,...
205,2017-02-28,0.174040,0.023948,0.0357,-0.0204,-0.0167,0.0004,-0.0160,0.0361,0.173640,0.023548
206,2017-03-31,0.030453,0.023550,0.0017,0.0113,-0.0333,0.0003,-0.0103,0.0020,0.030153,0.023250
207,2017-04-30,-0.007113,0.023485,0.0109,0.0072,-0.0213,0.0005,0.0048,0.0114,-0.007613,0.022985
208,2017-05-31,0.062052,0.042636,0.0106,-0.0252,-0.0375,0.0006,0.0150,0.0112,0.061452,0.042036


In [237]:
has_capm = 'HAS_ex ~ 1 + MF'
res_has_capm = smf.ols(has_capm, df2).fit()

has_f4 = 'HAS_ex ~ 1 + MF + SMB + HML + MOM'
res_has_f4 = smf.ols(has_f4, df2).fit()

rjr_capm = 'RJR_ex ~ 1 + MF'
res_rjr_capm = smf.ols(rjr_capm, df2).fit()

rjr_f4 = 'RJR_ex ~ 1 + MF + SMB + HML + MOM'
res_rjr_f4 = smf.ols(rjr_f4, df2).fit()

In [238]:
res_has_capm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 HAS_ex   R-squared:                       0.255
Model:                            OLS   Adj. R-squared:                  0.251
Method:                 Least Squares   F-statistic:                     71.06
Date:                Wed, 09 Mar 2022   Prob (F-statistic):           5.76e-15
Time:                        20:07:49   Log-Likelihood:                 264.88
No. Observations:                 210   AIC:                            -525.8
Df Residuals:                     208   BIC:                            -519.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0067      0.005      1.403      0.162      -0.003       0.016
MF             0.9132      0.108      8.430      0.000       0.700       1.127
==============================================================================
Omnibus:                        6.200   Durbin-Watson:                   2.152
Prob(Omnibus):                  0.045   Jarque-Bera (JB):                8.227
Skew:                           0.177   Prob(JB):                       0.0163
Kurtosis:                       3.902   Cond. No.                         22.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [239]:
res_has_f4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 HAS_ex   R-squared:                       0.261
Model:                            OLS   Adj. R-squared:                  0.247
Method:                 Least Squares   F-statistic:                     18.11
Date:                Wed, 09 Mar 2022   Prob (F-statistic):           9.45e-13
Time:                        20:07:51   Log-Likelihood:                 265.79
No. Observations:                 210   AIC:                            -521.6
Df Residuals:                     205   BIC:                            -504.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0061      0.005      1.262      0.209      -0.003       0.016
MF             0.8622      0.124      6.959      0.000       0.618       1.106
SMB            0.1324      0.156      0.849      0.397      -0.175       0.440
HML            0.1481      0.157      0.945      0.346      -0.161       0.457
MOM           -0.0647      0.097     -0.668      0.505      -0.256       0.126
==============================================================================
Omnibus:                        5.826   Durbin-Watson:                   2.129
Prob(Omnibus):                  0.054   Jarque-Bera (JB):                8.075
Skew:                           0.129   Prob(JB):                       0.0176
Kurtosis:                       3.925   Cond. No.                         37.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [240]:
res_rjr_capm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RJR_ex   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     11.42
Date:                Wed, 09 Mar 2022   Prob (F-statistic):           0.000866
Time:                        20:07:51   Log-Likelihood:                 243.16
No. Observations:                 210   AIC:                            -482.3
Df Residuals:                     208   BIC:                            -475.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0164      0.005      3.093      0.002       0.006       0.027
MF             0.4061      0.120      3.380      0.001       0.169       0.643
==============================================================================
Omnibus:                       24.339   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               81.666
Skew:                           0.362   Prob(JB):                     1.85e-18
Kurtosis:                       5.968   Cond. No.                         22.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [241]:
res_rjr_f4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RJR_ex   R-squared:                       0.105
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     6.041
Date:                Wed, 09 Mar 2022   Prob (F-statistic):           0.000129
Time:                        20:07:52   Log-Likelihood:                 249.25
No. Observations:                 210   AIC:                            -488.5
Df Residuals:                     205   BIC:                            -471.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0155      0.005      2.963      0.003       0.005       0.026
MF             0.4885      0.134      3.644      0.000       0.224       0.753
SMB           -0.3076      0.169     -1.823      0.070      -0.640       0.025
HML            0.4025      0.169      2.375      0.018       0.068       0.737
MOM            0.0129      0.105      0.123      0.902      -0.194       0.220
==============================================================================
Omnibus:                       17.022   Durbin-Watson:                   2.100
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               50.198
Skew:                           0.186   Prob(JB):                     1.26e-11
Kurtosis:                       5.366   Cond. No.                         37.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""